<a href="https://colab.research.google.com/github/PETERMWAURA254/Boston-Housing-Price-Prediction/blob/main/hate_speech_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn spacy transformers imbalanced-learn
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 115.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


 ### **Import** **libraries**

In [2]:
import pandas as pd
import numpy as np
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments


In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.head()

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
# dataset shape to know how many tweets in the datasets
print(f"num of tweets: {df.shape}")

# extract the text and labels
tweet = list(df['tweet'])
labels = list(df['class'])

num of tweets: (24783, 6)


 ### **Preprocess**  **dataset**

In [ ]:
# Preprocessing function
def preprocess_text(text):
    # Remove URLs, special characters, and extra spaces
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove non-alphabetic characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text.lower()  # Convert to lowercase
    # Tokenization and Lemmatization using spaCy
nlp = spacy.load('en_core_web_sm')
# Preprocessing function
def preprocess_text(text):
    # Remove URLs, special characters, and extra spaces
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove non-alphabetic characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text.lower()  # Convert to lowercase

# Tokenization and Lemmatization using spaCy
import spacy
nlp = spacy.load('en_core_web_sm')
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop])
  # Apply preprocessing
df['cleaned_tweet'] = df['tweet'].apply(preprocess_text)
df['lemmatized_tweet'] = df['cleaned_tweet'].apply(lemmatize_text)
df['lemmatized_tweet'] = df['cleaned_tweet'].apply(lemmatize_text)

In [ ]:
print(df.columns)

Index(['count', 'hate_speech_count', 'offensive_language_count',
       'neither_count', 'class', 'tweet', 'cleaned_tweet', 'lemmatized_tweet'],
      dtype='object')


### Prepare dataset for model training

In [ ]:
X = df['lemmatized_tweet']
y = df['class']  # Assuming label column indicates hate speech (0: non-hate, 1: hate)

###  Split data into training and testing *sets*

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Handling Imbalanced Data with SMOTE

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text into numerical features using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)  # Convert training text to TF-IDF vectors
X_test_tfidf = tfidf.transform(X_test)  # Convert test text to TF-IDF vectors

# Handle Class Imbalance using SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

### Logistic Regression model
    

In [ ]:
# Logistic Regression Model Training
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_smote, y_train_smote)

LogisticRegression(max_iter=1000)

In [ ]:
# Predictions
y_pred_logreg = logreg.predict(X_test_tfidf)

In [ ]:
# Evaluation
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_logreg))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.28      0.57      0.38       290
           1       0.96      0.85      0.90      3832
           2       0.78      0.90      0.84       835

    accuracy                           0.85      4957
   macro avg       0.67      0.78      0.71      4957
weighted avg       0.89      0.85      0.86      4957



### Load BERT

In [5]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW # Import AdamW from torch.optim instead
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Load and sample the dataset
df = pd.read_csv("train.csv").sample(n=3000, random_state=42)

# 2. Binary label: 1 = hate speech (class == 0), 0 = not hate
df['label'] = df['class'].apply(lambda x: 1 if x == 0 else 0)
df = df[['tweet', 'label']]

# 3. Train/test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['tweet'], df['label'], test_size=0.2, random_state=42
)

# 4. Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# 5. Dataset class
class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HateSpeechDataset(train_encodings, list(train_labels))
test_dataset = HateSpeechDataset(test_encodings, list(test_labels))

# 6. Load BERT
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 7. Setup
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
optimizer = AdamW(model.parameters(), lr=2e-5)

# 8. Training (1 epoch only)
model.train()
for epoch in range(1):  # ONE epoch
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch + 1} - Loss: {total_loss:.4f}")

# 9. Evaluation
model.eval()
test_loader = DataLoader(test_dataset, batch_size=32)
preds, true_labels = [], []

for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    preds.extend(predictions.cpu().numpy())
    true_labels.extend(batch['labels'].cpu().numpy())

print("\nClassification Report:")
print(classification_report(true_labels, preds, target_names=['not_hate', 'hate']))

# 10. Prediction function
def predict(texts):
    model.eval()
    if isinstance(texts, str):
        texts = [texts]

    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
    encodings = {k: v.to(device) for k, v in encodings.items()}

    with torch.no_grad():
        outputs = model(**encodings)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()
    labels = ['not_hate' if p == 0 else 'hate' for p in predictions]

    for i, text in enumerate(texts):
        print(f"\nText: {text}\n→ Prediction: {labels[i]}")

# 11. Example predictions
predict("I love your energy!")
predict("You are disgusting.")
predict(["You idiot,you are stupid"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Loss: 36.8784

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.94      1.00      0.97       564
        hate       0.00      0.00      0.00        36

    accuracy                           0.94       600
   macro avg       0.47      0.50      0.48       600
weighted avg       0.88      0.94      0.91       600



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Text: I love your energy!
→ Prediction: not_hate

Text: You are disgusting.
→ Prediction: not_hate

Text: You idiot,you are stupid
→ Prediction: not_hate
